In [288]:
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from config.get import cfg
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from TokenStandardScaler import TokenStandardScaler
print("===================================")     
from sklearn.model_selection import train_test_split
data = pd.read_csv(cfg['files']['preprocessed_data'],nrows=10000).drop(columns=["time"])

In [289]:
data = data.set_index(["cycle_id","token1","token2"])

In [291]:
a , b = train_test_split(data,train_size=0.8)

In [293]:
data.sample(frac=0.8)

baseAmount   quoteAmount    quotePrice   gasPrice  \
cycle_id token1 token2                                                       
1003     BPT    STA        0.121335   2342.851067  19308.897064  23.350000   
1001     WETH   BPT        0.960311      0.208190      0.216794  30.000001   
1004     USDC   LEND      89.365792    625.826412      7.002975  20.000000   
1005     WETH   DZAR       6.267027  27579.600000   4400.746736  22.000001   
1002     STA    WETH    7068.533306      0.254957      0.000036  32.000000   
...                             ...           ...           ...        ...   
1003     STA    WETH    4898.432444      0.397208      0.000081  17.999999   
1005     BPT    WETH       0.037081      0.162704      4.387798  22.550000   
                WETH       0.080000      0.405671      5.070883  52.999999   
1004     USDC   LEND     803.509762   5613.938670      6.986771  40.020001   
1006     WETH   BUIDL      0.019296     31.782277   1647.125601  20.000000   

                        gasValue  
cycle_id token1 token2            
1003     BPT    STA     0.007640  
1001     WETH   BPT     0.004868  
1004     USDC   LEND    0.004684  
1005     WETH   DZAR    0.015643  
1002     STA    WETH    0.005626  
...                          ...  
1003     STA    WETH    0.002445  
1005     BPT    WETH    0.007300  
                WETH    0.006481  
1004     USDC   LEND    0.034558  
1006     WETH   BUIDL   0.002648  

[8000 rows x 5 columns]

In [297]:
train_ix, test_ix = train_test_split(data.index.levels[0],train_size=0.8)
X_train = data.loc[train_ix]
X_test = data.loc[test_ix]

In [182]:
scaler   = TokenStandardScaler()
tX_train = scaler.fit_transform(X_train)
tX_test  = scaler.transform(X_test)

In [299]:
data.shape

(5000, 5)

In [284]:
def build_tensor(X_padded):
    N = len(X_padded.index.levels[0])
    N_TOKEN = 3 # cycle length
    K = 2       # quote price & gasPrice
    P = 600     # max time series length per cycle

    cycle_ids = np.zeros(N)
    tensor = np.zeros((N, N_TOKEN,P, K))
    i = 0
    for cycle_id, group in iter(X_padded.groupby("cycle_id")):
            token_map = group
            cycle_ids[i] =cycle_id
            tensor[i]    = group[["quotePrice","gasPrice"]].values.reshape((3, 600, 2))
            i+=1
            if i>=N-1 : break
    return cycle_ids[:-1],tensor[:-1] # last datapoint migh not be complete

In [279]:
train_padded = pad(tX_train)
test_padded = pad(tX_test)

In [287]:
train_ids , train_tensor = build_tensor(train_padded)
test_ids   , test_tensor = build_tensor(test_padded)

cfg['files']

{'cycles_in_Uniswap': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/cycles_in_Uniswap.json',
 'preprocessed_data': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/uniswap_data.csv',
 'raw_features': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/raw_features.npy',
 'raw_test_features': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/raw_test_features.npy',
 'raw_train_features': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/raw_train_features.npy',
 'test_ids': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/test_ids.npy',
 'train_ids': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/train_ids.npy',
 'encoded_test_features': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/encoded_test_features.npy',
 'encoded_train_features': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/encoded_train_features.npy',
 'cycle_ids': '/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/ML_features/cycle_ids.npy',
 'all_cycles': '/s

In [296]:
print("===================================")     
data = pd.read_csv(cfg['files']['preprocessed_data'],nrows=5000).drop(columns=["time"])
data = data.set_index(["cycle_id","token1","token2"])


# train test split
print("splitting")
train_ix, test_ix = train_test_split(data.index.levels[0],train_size=0.8)
X_train = data.loc[train_ix, :]
X_test = data.loc[test_ix, :]
print(f"Shapes : X_train={X_train.shape}, X_test={X_test.shape}")

splitting
Shapes : X_train=(3306, 5), X_test=(1694, 5)


In [301]:
train_ix, test_ix = train_test_split(data.index.levels[0])
train_df = data.loc[train_ix]
test_df = data.loc[test_ix]

In [302]:
train_df.shape

(3306, 5)

In [303]:
train_ix

Int64Index([1003, 1000, 1002], dtype='int64', name='cycle_id')

In [304]:
test_df

baseAmount  quoteAmount  quotePrice   gasPrice  \
cycle_id token1 token2                                                     
1001     WETH   BPT         5.899406     0.028860    0.004892  17.999999   
                BPT         0.100000     0.001160    0.011598  16.999999   
                BPT         0.050000     0.000559    0.011189  16.999999   
                BPT         1.000000     0.008956    0.008956  19.000001   
                BPT         0.213296     0.010674    0.050041  60.999999   
...                              ...          ...         ...        ...   
         STA    WETH    12316.876138     0.492447    0.000040  40.000000   
                WETH     8777.103031     0.356534    0.000041  23.928320   
                WETH    10283.666585     0.424396    0.000041  20.999999   
                WETH     7738.232655     0.322581    0.000042  18.749999   
                WETH     1242.962529     0.050000    0.000040  28.000000   

                        gasValue  
cycle_id token1 token2            
1001     WETH   BPT     0.002471  
                BPT     0.002129  
                BPT     0.001874  
                BPT     0.002380  
                BPT     0.007058  
...                          ...  
         STA    WETH    0.007911  
                WETH    0.004198  
                WETH    0.002853  
                WETH    0.003294  
                WETH    0.003803  

[1694 rows x 5 columns]